In [ ]:
import logging
import os
from dotenv import load_dotenv

load_dotenv()
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

USER_ID = os.getenv("DEEZER_USER_ID")
logging.info(f"User ID: {USER_ID}")

In [ ]:
from pathlib import Path
from deezerus import get_track_list

df_tracks = get_track_list(USER_ID, full_version=False)
df_tracks.to_csv(Path.home() / "Downloads" / "track_list.csv", index=False)

df_tracks.head()

In [ ]:
import pandas as pd
import requests
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# --- Fonctions de base (optimisées) ---


def _telecharger_audio(url):
    try:
        if isinstance(url, str) and url.startswith("http"):
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            return r.content if "audio" in r.headers.get("Content-Type", "") else None
    except requests.RequestException:
        return None


def obtenir_genres_en_lot(urls, batch_size=8, workers=10):
    classifier = pipeline(
        "audio-classification", model="dima806/music_genres_classification"
    )

    print("Étape 1: Téléchargement des fichiers audio...")
    with ThreadPoolExecutor(max_workers=workers) as executor:
        contenus = list(tqdm(executor.map(_telecharger_audio, urls), total=len(urls)))

    donnees_valides = [(i, audio) for i, audio in enumerate(contenus) if audio]
    if not donnees_valides:
        print("Aucun fichier audio valide trouvé.")
        return ["Erreur"] * len(urls)

    indices, audios_valides = zip(*donnees_valides)

    print("\nÉtape 2: Classification des genres...")
    predictions = list(
        tqdm(
            classifier(list(audios_valides), batch_size=batch_size),
            total=len(audios_valides),
        )
    )

    resultats = {indices[i]: p[0]["label"] for i, p in enumerate(predictions) if p}

    return [resultats.get(i, "Erreur") for i in range(len(urls))]


# --- Fonction principale pour le DataFrame ---


def classifier_tous_les_genres(df):
    """
    Analyse le DataFrame, classifie les genres pour toutes les musiques,
    et retourne le DataFrame mis à jour.
    """
    # Crée la colonne 'genre' si elle n'existe pas, avec une valeur par défaut
    if "genre" not in df.columns:
        df["genre"] = None

    # Sélectionne les musiques qui n'ont pas encore de genre valide
    # On utilise .index pour garder une référence aux lignes originales
    lignes_a_classifier = df[df["genre"].isnull()].index

    if lignes_a_classifier.empty:
        print("Toutes les musiques ont déjà un genre. Aucune action requise.")
        return df

    print(f"Il y a {len(lignes_a_classifier)} musiques à classifier.")

    # Récupère la liste des URLs à traiter
    urls_a_traiter = df.loc[lignes_a_classifier, "preview"].tolist()

    # Lance le traitement par lot sur ces URLs
    genres_predits = obtenir_genres_en_lot(urls_a_traiter)

    # Met à jour la colonne 'genre' du DataFrame original aux bons endroits
    df.loc[lignes_a_classifier, "genre"] = genres_predits

    print("\n✅ Classification terminée. Le DataFrame a été mis à jour.")
    return df

In [ ]:
df_tracks_complet = classifier_tous_les_genres(df_tracks[10:20])

display(df_tracks_complet[["title", "artist", "genre"]].head(10))

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import requests

# Assurez-vous que votre DataFrame 'df_tracks' est chargé avant ce code
# df_tracks = ...

print("Création de la pipeline de classification de genre...")
genre_classifier = pipeline(
    "audio-classification", model="dima806/music_genres_classification"
)
print("Pipeline créée.")

# --- NOUVELLE LOGIQUE PLUS ROBUSTE ---

print("\n⏳ Téléchargement des aperçus audio...")

# On ne stocke que les données valides et leurs indices d'origine
valid_audio_data = []
original_indices = []

for index, url in tqdm(enumerate(df_tracks["preview"]), total=len(df_tracks)):
    if isinstance(url, str) and url.startswith("http"):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            # On ajoute les données et l'indice si le téléchargement réussit
            valid_audio_data.append(response.content)
            original_indices.append(index)
        except requests.RequestException:
            # Si le téléchargement échoue, on ne fait rien
            pass

print(f"\n✅ {len(valid_audio_data)} aperçus audio valides trouvés.")
print("\n🚀 Prédiction des genres par lots sur le CPU...")

# On passe UNIQUEMENT la liste des données valides à la pipeline.
results_list = genre_classifier(valid_audio_data, batch_size=8)

# On initialise la colonne de genre avec une valeur par défaut
genres = ["Genre inconnu"] * len(df_tracks)

# On réinsère les résultats aux bons endroits grâce à la liste des indices
for i, result in enumerate(results_list):
    # L'indice dans le DataFrame d'origine
    original_index = original_indices[i]
    if result and isinstance(result, list) and len(result) > 0:
        # On place le genre prédit à la bonne position
        genres[original_index] = result[0]["label"]
    else:
        genres[original_index] = "Erreur de prédiction"


# On ajoute la nouvelle colonne de genres au DataFrame
df_tracks["genre"] = genres

print("\n--- DataFrame final avec les Genres ---")
# Affiche les colonnes pertinentes, y compris pour les genres inconnus
print(df_tracks[["title", "artist", "genre"]].head(10))

# On sauvegarde les résultats dans le cache pour ne pas avoir à le refaire
df_tracks[["id", "genre"]].to_csv("track_genres_pipeline.csv", index=False)